In [1]:
import pandas as pd
df = pd.read_excel('../../data/CleanedStatements/fully_catized_encoded.xlsx')
import nltk
import re
import spacy
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
# from contractions import CONTRACTION_MAP
import unicodedata

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

KeyboardInterrupt: 

In [ ]:
# nlp = spacy.load('en_core_web_lg')

In [ ]:
# Find below my suggestions:

# 1. Consider the frequency of words, remove any words that occur less than 5 or 10 times,
# this helps us get rid of unique words that are not used frequently, this way our model 
# will not focus on those words.

# 2. W.r.t the above point, try removing the words that are too frequent like the one that 
# occurs more then 100-120 times, this gets rid of the higher frequency words. These 
# threshold values are configurable and solely depends on the use case, there is no right or 
# wrong way to do it.

# 3. Try removing Stop words.

# 4. Try Lemmatisation, it is recommended you experiment with the Lemmatisation after stop 
# words removal and then try the frequency removal method.

# Try these methods and let me know if it worked for you, if not feel free to reach out in 
# the response section, I’ll try to respond sooner.

In [ ]:
df[(-2.5<df['Amount']) & (df['Amount']<-1)]['Amount'].plot.hist(bins=54,alpha=0.5)
import matplotlib.pyplot as plt

# # Creating histogram
# fig, ax = plt.subplots(figsize =(10, 7))
# ax.hist(df[(-40<df['Amount']) & (df['Amount']<-3)]['Amount'], bins = [0, 25, 50, 75, 100])
  
# # Show plot
# plt.show()

In [ ]:
df[(-40<df['Amount']) & (df['Amount']<-3)]['Amount'].plot.box()

In [ ]:
import seaborn as sns

In [ ]:
fig = plt.subplots(figsize=(12,6))
sns.barplot(df['Category'],df[abs(df['Amount']) < 400]['Amount'])
plt.xticks(rotation = 90) 
plt.title('Spending per category')
plt.show()

In [ ]:
spends = df[df['Amount'] < 0]['Amount'].apply(lambda x: -1*x)
cats = df[df['Amount'] < 0]['Category'].copy()

In [ ]:
len(spends)

In [ ]:
fig = plt.subplots(figsize=(12,6))
plt.pie(spends)
plt.xticks(rotation = 90) 
plt.title('Spending per category')
plt.show()

In [ ]:
tokenizer

In [ ]:
import pandas as pd
df = pd.read_excel('../data/CleanedStatements/fully_catized_encoded.xlsx')
import nltk
import re
import spacy
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from nltk.tokenize.toktok import ToktokTokenizer
# from contractions import CONTRACTION_MAP
import unicodedata

from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder
# from tensorflow.keras.preprocessing.sequence import pad_sequences

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def data_corpus_create(content):
    data_corpus=set()
    for row in content:
        for word in row.split(" "):
            if word not in data_corpus:
                data_corpus.add(word)
    return sorted(data_corpus)

lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('omw-1.4')
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

content = df['Content'].copy()

# build corpus, get count of each word
nltk.FreqDist(content[0].split(' '))

content1 = content.apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", " ", x) )
content2 = content1.apply(lambda x: x.lower())
content3 = content2.apply(lambda x: x.strip())
# create corpus with frequency counts to see what can stay and what can depart
content4 = pd.Series(sorted(content3))
# Create a corpus for all the words
content5 = content4.apply(remove_stopwords)

content6 = content5.apply(lambda x: lemmatizer.lemmatize(x))
content7 = content6.apply(lambda x: re.sub("\d+","",x))
cont = content7.apply(lambda x: re.sub("  "," ",x))
cont = cont.apply(lambda x: x.strip())
cont = cont.apply(lambda x: re.sub("  "," ",x))

data_corpus = data_corpus_create(cont)

# Create dictionary for each unique word, += 1 for each new encounter
d = {}
for row in cont:
    for word in row.split(" "):
        if word not in d.keys():
            d[word] = 1
        else:
            d[word] +=1

word_freq_counts = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

ref = list(d.values())

def remove_dict_dat(sentence):
    s = sentence.split(" ")
    for word in s:
        if d[word] < 5:
            s.remove(word)
        elif d[word] > 19:
            s.remove(word)
    return " ".join(s)

cont2 = cont.apply(remove_dict_dat)

data_corpus=data_corpus_create(cont2)

cont2 = cont2.apply(lambda x: x.strip())
cont2 = cont2.apply(lambda x: lemmatizer.lemmatize(x))

# converting text to integers
token_docs = [list(set(doc.split(" "))) for doc in cont2]
all_tokens = set([word for sentence in token_docs for word in sentence])
word_to_idx = {token:idx+1 for idx, token in enumerate(all_tokens)}

# converting the docs to their token ids
X = np.array([[word_to_idx[token] for token in token_doc] for token_doc in token_docs], dtype=object)

# padding the sequences
X_padded = pad_sequences(X, padding="post")

# converting to pandas df
X_df = pd.DataFrame(X_padded)

X_df

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
sentences = pd.Series(token_docs)
mlb = MultiLabelBinarizer()
mlb_df = df.join(pd.DataFrame(mlb.fit_transform(sentences),columns=mlb.classes_))
mlb_df

In [ ]:
X

In [ ]:
# change numeric columns to str for the transaction text
lis = []

for x in X_df.columns:
    lis.append('word'+str(x))
X_df.columns = lis

# combine original dataframe to the parameterized content dataframe
dff = pd.concat([df,X_df],axis=1)

In [ ]:
# os.chdir('..')
from auto_budget import bank_functions_NWFCU as nw
ref = nw.ref_dictionaries(dff)

In [ ]:
# just ordinal encoding, about 63% accuracy no cross-validation

# change numeric columns to str for the transaction text
lis = []

for x in X_df.columns:
    lis.append('word'+str(x))
X_df.columns = lis

# combine original dataframe to the parameterized content dataframe
dff = pd.concat([df,X_df],axis=1)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# separate predictors and response
X = dff.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
y = dff['Category_Cat'].copy()

# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
predictions = dectree.predict(X_test)
sum(predictions == y_test.array) / len(predictions)

In [ ]:
# one hot encoding but with redundancy

enc = OneHotEncoder(handle_unknown ='ignore')
enc_df = pd.DataFrame(enc.fit_transform(X_df).toarray())

# change numeric columns to str for the transaction text
lis = []

for x in enc_df.columns:
    lis.append('word'+str(x))
enc_df.columns = lis

# combine original dataframe to the parameterized content dataframe
dff = pd.concat([df,enc_df],axis=1)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# separate predictors and response
X = dff.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
y = dff['Category_Cat'].copy()

# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
predictions = dectree.predict(X_test)
sum(predictions == y_test.array) / len(predictions)

In [ ]:
# one hot encoding without redundancy

from sklearn.preprocessing import MultiLabelBinarizer
sentences = pd.Series(token_docs)
mlb = MultiLabelBinarizer()
mlb_df = df.join(pd.DataFrame(mlb.fit_transform(sentences),columns=mlb.classes_))

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# separate predictors and response
X = mlb_df.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
y = mlb_df['Category_Cat'].copy()

# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
predictions = dectree.predict(X_test)
sum(predictions == y_test.array) / len(predictions)

In [ ]:
# no transaction words at all

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# separate predictors and response
X = df.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
y = df['Category_Cat'].copy()

# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
predictions = dectree.predict(X_test)
sum(predictions == y_test.array) / len(predictions)

In [ ]:
# figure out how to use index-based words in sklearn 
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#tokenizing-text-with-scikit-learn

import seaborn as sns
import matplotlib.pyplot as plot
import warnings as w
%matplotlib inline

# supress warnings
w.filterwarnings('ignore')

num_simulations = 10000
# results = pd.DataFrame(columns=['trial','no_words','words_ordinal','words_onehot'])
results = pd.DataFrame()
dectree = DecisionTreeClassifier()

def feature_selection(X,y):
    # initial fit
    dectree.fit(X, y)
    model = SelectFromModel(dectree, prefit=True)
    X_new = model.transform(X)
    return X_new

def decision_tree_results(prepped_df, featselect=False, dectree):
    """runs split decision tree model and returns percent of matches"""
    # separate predictors and response
    X = prepped_df.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
    y = prepped_df['Category_Cat'].copy()
    
    if featselect:
        X_new = feature_selection(X, y)
    else:
        X_new = X.copy()

    # test train split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # dectree = DecisionTreeClassifier()
    dectree.fit(X_train, y_train)
    predictions = dectree.predict(X_test)
    return (sum(predictions == y_test.array) / len(predictions)) * 100

# run 1,000 simulations for each dataset, and see which has the best misclassification rate

for i in range(num_simulations):
    # no_words = decision_tree_results(df)
    # words_ordinal = decision_tree_results(dff)
    words_onehot = decision_tree_results(mlb_df, dectree)
    words_onehot_featselect = decision_tree_results(mlb_df,True,dectree)
    #results = results.append({'trial':i,'no_words':no_words,'words_ordinal':words_ordinal,'words_onehot':words_onehot}, \
    #                         ignore_index=True)
    results = results.append({'trial':i,'words_onehot':words_onehot,'words_onehot_featselect':words_onehot_featselect}, \
                            ignore_index=True)

# pivot dataframe to use seaborn hue

results_melted = pd.melt(results,id_vars=['trial'])

# plot results in graph
plt.figure(figsize=(16,10))
sns.lineplot(x='trial',y='value',hue='variable',data=results_melted)
plt.title('Comparison of datasets in decision tree')

In [ ]:
mlb_df[mlb_df.columns[26:]].sum()

In [ ]:
# feature selection for the mlb_df

In [ ]:
mlb_df.head()

In [ ]:
# One hot with feature selection

from sklearn.feature_selection import SelectFromModel

# separate predictors and response
X = mlb_df.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
y = mlb_df['Category_Cat'].copy()

# decision tree model
dectree = DecisionTreeClassifier()

# initial fit
dectree.fit(X, y)
model = SelectFromModel(dectree, prefit=True)
X_new = model.transform(X)

# test train split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=42)

# decision tree model
dectree.fit(X_train, y_train)
predictions = dectree.predict(X_test)
(sum(predictions == y_test.array) / len(predictions)) * 100

In [ ]:
results['featselect'] = words_onehot_featselect >= results.words_onehot
sum(results.featselect) / len(results)

In [ ]:
results

In [ ]:
import pandas as pd
os.chdir('code')
mlb_df = pd.read_excel('mlb_df.xlsx')
mlb_df.head()

In [ ]:
import os
os.getcwd()

In [ ]:
# figure out how to use index-based words in sklearn 
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#tokenizing-text-with-scikit-learn

import seaborn as sns
import matplotlib.pyplot as plt
import warnings as w
%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

# supress warnings
w.filterwarnings('ignore')

num_simulations = 10000
# results = pd.DataFrame(columns=['trial','no_words','words_ordinal','words_onehot'])
results = pd.DataFrame()
dectree = DecisionTreeClassifier()
ranfore = RandomForestClassifier(n_estimators=17)

def feature_selection(X,y):
    # initial fit
    dectree.fit(X, y)
    model = SelectFromModel(dectree, prefit=True)
    X_new = model.transform(X)
    return X_new

def model_evaluation(prepped_df, model):
    """runs split decision tree model and returns percent of matches"""
    # separate predictors and response
    X = prepped_df.drop(['Purchase Date','Purchase Time','Payment_Method','Verification Date', 'Content', 'Category','PT_total_seconds','Payment_Method_Cat', 'Category_Cat', 'Content'], axis=1)
    y = prepped_df['Category_Cat'].copy()
    
    # Feature selection
    X_new = feature_selection(X, y)

    # test train split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return (sum(predictions == y_test.array) / len(predictions)) * 100

# run 1,000 simulations for each dataset, and see which has the best misclassification rate

for i in range(num_simulations):
    # no_words = decision_tree_results(df)
    # words_ordinal = decision_tree_results(dff)
    decisiontree = model_evaluation(mlb_df, dectree)
    randomforest = model_evaluation(mlb_df,ranfore)
    #results = results.append({'trial':i,'no_words':no_words,'words_ordinal':words_ordinal,'words_onehot':words_onehot}, \
    #                         ignore_index=True)
    results = results.append({'trial':i,'Decision Tree':decisiontree,'Random Forest':randomforest}, \
                            ignore_index=True)

# pivot dataframe to use seaborn hue

results_melted = pd.melt(results,id_vars=['trial'])

# plot results in graph
plt.figure(figsize=(16,10))
sns.lineplot(x='trial',y='value',hue='variable',data=results_melted)
plt.title('Comparison of models')

In [ ]:
# upload historic data and train decision tree model 
# upload new data, clean, encode and categorize using model
# review decision tree's categorization against my own
# add new encoded data to historic data
# aggregate new data by category and upload as new sheet to budget